# Phase de Nettoyage des données

Dans ce notebook seront présentées les démarches effectuées pour la récupération, le tri et le nettoyage des données mises à notre disposition.

Nous avons au fil de notre projet eu besoin d'obtenir des données de plusieurs sources différentes:

- Des données complètes obtenues via la plateforme Open Data de [l'European Drug Agency](https://www.euda.europa.eu/index_en), qui se sont avérées parfois complètes, et parfois parcellaires. Certaines tables on notamment nécessité un gros travail de merge et de nettoyage à cause de la présence de plusieurs variables manquantes. Nous détaillerons dans une partie détaillée les choix méthodologiques implémentés dans le nettoyage de ces données.

- Des données sociales et économiques obtenues via le portail [d'Eurostat](https://ec.europa.eu/eurostat/fr/), ces dernières ont nécessité un léger travail de merge détaillé dans la dernière partie

- Enfin, des données sur la légalisation du cannabis dans chaque pays, scrapées depuis Wikipedia. Toutes les décisions prises lors de la récupération de ces données est détaillée point par point dans la partie suivante.

## 1. Nettoyage des données scrapées depuis Wikipedia

On commence par générer notre dataframe à partir du scraping des données de [l'article Wikipedia sur la légalisation du Cannabis par Pays](https://en.wikipedia.org/wiki/Legality_of_cannabis). Le code détaillé et commenté se trouve dans le fichier `scraping.py`, on se contente ici d'appeler nos fonctions.


In [258]:
from Data.scraping import scraping as s

url = "https://en.wikipedia.org/wiki/Legality_of_cannabis"

table = s.scrape_second_wikipedia_table(url)
data, header = s.table_to_list(table)
legal_pays = s.make_pandas(data, header)

legal_pays.head()

,Recreational,Medical,Notes
Afghanistan,Illegal,Illegal,Main article: Cannabis in Afghanistan Producti...
Albania,Illegal,Legal,Main article: Cannabis in Albania Prohibited b...
Algeria,Illegal,Illegal,Main article: Cannabis in Algeria
Andorra,Illegal,Illegal,Main article: Cannabis in Andorra
Angola,Illegal,Illegal,Main article: Cannabis in Angola


On remarque alors que notre Dataframe contient plusieurs éléments qui nécéssitent d'être nettoyés, modifiés, voire supprimés. Dans l'ordre on peut citer:

1. **Supprimer la dernier colonne concernant les "Notes"**, qui peuvent nous être utile à des fins d'interprétation, ou encore pour aider à la classification ou à l'interprétation plus tard, mais qui en l'état seront inutiles à la statistique descriptive ou à la construction de modèles plus tard.

2. **Eliminer de notre dataframe les pays non-européens**, puisque les données de l'EUDA ne s'intéresse qu'au cas européeen, et que c'est là le coeur de notre sujet.

3. Enfin, il nous faudra **regrouper les différentes états des variables sous un format clair** En l'état, on dispose pour chaque pays d'une description parfois unique et idividuelle de la législation dans le pays donné. L'idéal serait de ne garder qu'une variable binaire: Légal/Illégal. Cependant la réalité est plus complexe dans certains pays, et il nous faudra alors faire des choix, et les justifier, pour essayer de ne garder qu'un nombre limité d'état pour chaque variable.

### Suppression de la colonne des Notes

Cela n'est pas bien compliqué, on utilise simplement les outils de la librairie Pandas:

In [259]:
import pandas as pd

legal_pays_clean = legal_pays.drop(columns = "Notes")

legal_pays_clean.head()

,Recreational,Medical
Afghanistan,Illegal,Illegal
Albania,Illegal,Legal
Algeria,Illegal,Illegal
Andorra,Illegal,Illegal
Angola,Illegal,Illegal


### Tri pour ne garder que les pays Européens

Cette opération n'est elle aussi pas très compliquée, on va se servir d'une table de correspondance scrapée sur le [site national de Statistiques Candiennes](https://www23.statcan.gc.ca/imdb/p3VD.pl?Function=getVD&TVD=1312110) (ce afin d'avoir le nom des pays en anglais), qui nous sera d'ailleurs aussi utile plus tard afin de faire correspondre chaque pays de l'Union européenne avec son code ISO 3316.

In [260]:
from Data.scraping import scraping as s

url = "https://www23.statcan.gc.ca/imdb/p3VD.pl?Function=getVD&TVD=1312110"

table = s.scrape_first_table(url)
data, header = s.table_to_list(table)
table_corres = s.make_pandas(data)

table_corres.head()

,0,1
Belgium,BE,BEL
Bulgaria,BG,BGR
Croatia,HR,HRV
Cyprus,CY,CYP
Czechia,CZ,CZE


On doit néanmoins ajouter à la main certains pays non-membres de l'Union Européenne, mais présent dans les données que nous avons à disposition (souvent des pays candidats comme la Turquie):

In [261]:
import pandas as pd

table_corres.loc["Austria"] = ["AT","AUT"]
table_corres.loc["Türkiye"] = ["TR","TUR"]
table_corres.loc["Norway"] = ["NO","NOR"]

table_corres.tail()

,0,1
Spain,ES,ESP
Sweden,SE,SWE
Austria,AT,AUT
Türkiye,TR,TUR
Norway,NO,NOR


On effectue ensuite le tri des pays européens selon la manière suivante, en supprimant de la table principale toutes les lignes qui n'ont pas d'équivalent dans la table de correspondance.

In [262]:
# On renomme certains pays pour correspondre aux données de l'EUDA 
legal_pays_clean.rename(index={"Turkey":"Türkiye"}, inplace=True)
legal_pays_clean.rename(index={"Czech Republic":"Czechia"}, inplace=True)

for index in legal_pays_clean.index:
    if index in table_corres.index:
        pass
    else:
        legal_pays_clean = legal_pays_clean.drop(index = index)

legal_pays_clean.head()


,Recreational,Medical
Austria,Possession for personal use decriminalized as ...,"Dronabinol, Sativex, Nabilone[25]"
Belgium,Decriminalized up to 3 g (1⁄10 oz) or cultivat...,Sativex[31]
Bulgaria,Illegal,Illegal
Croatia,Decriminalized[71],Legal[72]
Cyprus,Illegal,Legal[73]


### Recodage de chaque élément dans une variable ternaire

Il nous faut désormais faire un travail de recodage des informations contenues dans le tableau, afin de ne garder au final qu'une variable avec trois états possibles:

- `Legal` si la consommation, la possession et la vente sont légales et encadrées dans le pays
- `Illegal` si la consommation, la possession et la vente sont illégale et passibles de poursuites judiciaires
- `Decriminalized` si la consommation et la possession sont décriminalisées, mais la vente est illégale. On ne fait pas ici de différence entre la pénalisation ou non du cannabis (la seule différence étant que dans le cas ou ce dernier est pénalisé, la consommation et la possession sont toujours passibles d'une amende, mais pas de poursuites légales)

L'information est souvent contenue en clair dans notre tableau, mais perdue au milieu de précisions et de nuances propres à chaque pays. Par souci de simplicité, on appliquera les règles suivantes afin de recoder efficacement le tableau:

- **On ignorera toute limite de poids ou de nombres de plants**: Par exemple dans le cas de la République Tchèque, la possession de cannabis est décriminalisée jusqu'à 10g, et illégale au delà. On la considère donc simplement comme décriminalisée.

- **Dans le domaine médical, l'autorisation du Sativex vaut pour légalisation, celle du CBD non**: En effet, le Sativex contient du THC tout comme certains autres médicaments dérivés de cannabis, ce qui n'est pas le cas pour le CBD (ou alors en quantité très faible). En outre, au vu du fait que le CBD soit légal dans la plupart des pays, considérer son autorisation comme valant légalisation aurait pour conséquence d'appauvrir largement nos données.

- **On se tiendra à la stricte législation théorique, sans prendre en compte pour l'instant son application pratique**, cela aussi par souci de simplicité car la situation de chaque pays est unique et complexe. Par exemple, en Finlande, la possession et la consommation de cannabis est illégale *stricto sensu* et peut valoir une peine de prison. En réalité, les consommateurs s'en tirent souvent avec une simple amende; ce qui serait presque équivalent *en pratique* à une décriminalisation. Le fait est qu'il nous est impossible de trancher pour chaque pays à quel moment s'arrête la théorie et commence la pratique. Nous n'avons en effet ni la compétence sur le sujet, ni l'autorité nécessaire pour effectuer ce genre d'arbitrage, ce qui nous amène à ne prendre en compte que la législation en vigueur, et non son application réelle.
En revanche, il pourra nous être utile de tenir compte de ces différences d'application de la loi lors de l'analyse que nous ferons de nos données, mais pas dans la construction de celles-ci.

En prenant cela en compte, nous pouvons recoder le tableau de la manière suivante:

In [263]:
# Recodage des information selon les règles fixées plus haut

for index in legal_pays_clean.index:

    content = legal_pays_clean.loc[index].values.flatten().tolist()
    
    for i in range(len(content)):
        if "Legal" in content[i] or "Sativex" in content[i]:
            content[i] = "Legal"
        elif "Illegal" in content[i] or "CBD" in content[i]:
            content[i] = "Illegal"
        elif "Decriminalized" in content[i] or "decriminalized" in content[i]:
            content[i] = "Decriminalized"

    legal_pays_clean.loc[index] = content

legal_pays_clean

,Recreational,Medical
Austria,Decriminalized,Legal
Belgium,Decriminalized,Legal
Bulgaria,Illegal,Illegal
Croatia,Decriminalized,Legal
Cyprus,Illegal,Legal
Czechia,Decriminalized,Legal
Denmark,Illegal,Legal
Estonia,Decriminalized,With a special permit[88]
Finland,Illegal,Legal
France,Illegal,"A two-year trial program involving 3,000 patie..."


Puisqu'il reste après filtrage encore quelques valeurs qui n'ont pas de définition claire, nous n'avons d'autre choix que de les recoder à la main, en justifiant bien évidemment nos choix à chaque étape:

- En Estonie, l'usage du cannabis pour la médecine est autorisé avec "un permis spécial". Dans les faits, une recherche plus détaillée sur ce permis nous apprend qu'il n'est en réalité jamais délivré et qu'un seul patient a ce jour a été autorisé à recevoir du Sativex. Cependant, pour être cohérent avec notre règle fixée plus haut de ne prendre en compte que la législation théorique en vigueur, on choisit de considérer l'usage comme Légal, puisqu'il est théoriquement possible.

- En France, l'usage cannabis thérapeutique est aujourd'hui sujet d'expérimentation dans certains cas précis et très encadrés, mais reste en théorie illégal. On choisit donc de le coder comme étant illégal puisque les expérimentations encadrées n'ouvrent pas à une possibilité d'utilisation pour la population générale.

- En Roumanie, l'usage thérapeutique de dérivés de cannabis avec une teneur en THC inférieure à 0,2% est autorisée. Etant donnée que la plupart des pays (dont la France) autorisent le CBD à condition qu'il ne contienne pas plus de 0,3% de THC, on choisit de considérer l'usage thérapeutique en Roumanie comme illégal, puisqu'on s'est fixé comme règle plus haut que la légalisation du CBD ne valait pas celle du cannabis.

In [264]:
# Recodage à la main des informations plus ambigues

legal_pays_clean.loc['France','Medical'] = 'Illegal'
legal_pays_clean.loc['Estonia','Medical'] = 'Legal'
legal_pays_clean.loc['Romania','Medical'] = 'Illegal'

legal_pays_clean.head()

,Recreational,Medical
Austria,Decriminalized,Legal
Belgium,Decriminalized,Legal
Bulgaria,Illegal,Illegal
Croatia,Decriminalized,Legal
Cyprus,Illegal,Legal


On obtient donc finalement un tableau complètement nettoyé et exploitable pour nos analyses futures !

## 2. Nettoyage des tables de soin fournies par l'EUDA

Ces tables proviennent aussi de l'Open Data de [l'EUDA](https://www.euda.europa.eu), mais contrairement aux données déjà téléchargées par ailleurs (en format csv), ces dernières sont des tables bien plus morcelées et parcellaires. En plus d'avoir le bon goût de n'être disponibles qu'au format excel, elles vont nécessiter un gros travail de nettoyage.

### 2.1 Importation des tables

On commence par l'importation des données **générales** sur les **soins et traitements** liés à la consommation de drogue.
On utilise le module **Pandas** pour passer de fichier .xslx (Excel) à des tables Pandas.
Par ailleurs, on ne garde à chaque fois que les colonnes qui nous intéressent en vue d'un **Merge**.

In [265]:
import pandas as pd
import openpyxl
import warnings

warnings.filterwarnings('ignore')

### On commence par les donnees generales
total = "/home/onyxia/work/Projet_Python_2A/Data/EUDA/Cannabis détaillé/jamais_traite_entrants/drogues_entrants.xlsx"
df1 = pd.read_excel(total, header=3)

# On ne garde que les colonnes qui nous intéressent
colonnes = ['Country', 'Year of Treatment', 'All substances', 'Cannabis']
df1 = df1[colonnes]

# On procede de la meme maniere pour les donnees concernant les personnes deja et jamais traitees auparavant
deja = "/home/onyxia/work/Projet_Python_2A/Data/EUDA/Cannabis détaillé/jamais_traite_entrants/deja_traite_entrants.xlsx"
df2 = pd.read_excel(deja, header=3)

colonnes = ['Country', 'Year of Treatment', 'All substances', 'Cannabis']
df2 = df2[colonnes]

jamais = "/home/onyxia/work/Projet_Python_2A/Data/EUDA/Cannabis détaillé/jamais_traite_entrants/jamais_traite_entrants.xlsx"
df3 = pd.read_excel(jamais, header=3)

colonnes = ['Country', 'Year of Treatment', 'All substances', 'Cannabis']
df3 = df3[colonnes]


### 2.2 Premier Merge : table_base
Une fois les tables préparées, on les **Merge** pour créer une unique table : *table_base*.
Celle-ci nous servira de base pour effectuer un **Merge** de plus grande ampleur afin d'obtenir une grande table compilant toutes les données disponibles ici sur le sujet des **soins et traitements** liés à la consommation de drogues.

Pour éviter des problèmes de **doublons dans le nom des colonnes** au moment du *Merge* final, on modifie certains de ces noms.

In [266]:
### On effectue le merge (left join) afin de grouper les données par pays


## On commence par un merge "intermédiaire"
# On commence par supprimer la colonne "year of treatment" pour df 2 car cette colonne apparaît dans les deux tables
df2 = df2.drop(columns=["Year of Treatment"])
intermediaire = pd.merge(df1, df2, how="left", on="Country")
# Il faut renommer certaines colonnes (noms en doublon)
intermediaire = intermediaire.rename(columns={
    "Country": "Country",
    "Year of Treatment": "Year of Treatment",
    "All substances_x": "All substances (global)",
    "Cannabis_x": "Cannabis (global)",
    "All substances_y": "All substances (deja traite.es)",
    "Cannabis_y": "Cannabis (deja traite.es)"
})


## On reproduit les mêmes étapes mais en "mergeant" intermediaire et df3 pour finaliser cette table de BaseException
# On commence par supprimer la colonne "year of treatment" pour df 2 car cette colonne apparaît dans les deux tables
df3 = df3.drop(columns=["Year of Treatment"])
table_base = pd.merge(intermediaire, df3, how="left", on="Country")
# Il faut renommer certaines colonnes (noms en doublon)
table_base = table_base.rename(columns={
    "Country": "Country",
    "Year of Treatment": "Year of Treatment",
    "All substances (global)": "All substances (global)",
    "Cannabis (global)": "Cannabis (global)",
    "All substances (deja traite.es)": "All substances (deja traite.es)",
    "Cannabis (deja traite.es)": "Cannabis (deja traite.es)",
    "All substances": "All substances (jamais traite.es)",
    "Cannabis": "Cannabis (jamais traite.es)",
})

# On enlève les lignes en trop à la fin (vides ou inutiles)
table_base = table_base.drop(table_base.index[29:32:])


### 2.3 Vérifications préalables au Merge final

#### 2.3.1 Première vérification: présence de la colonne de merge

Avant de **Merge** toutes les tables disponibles sur le sujet, on vérifie que la colonne **"Country"** sur la base de laquelle se fera le *Merge* existe bien dans tous les *Dataframes*.

Procéder à une vérification préalable nous évite de potentiels soucis lors du *Merge* final, soucis dont nous pourrions passer à côté et ne pas les remarquer du fait du nombre important de colonnes et de tables à *Merge*. 

Par ailleurs, le **module os** nous permet d'inclure dans des boucles *for* des chemins de fichiers et ainsi généraliser la procédure et **alléger le code**.

In [267]:
warnings.filterwarnings('ignore')

# Etape préalable : avant d'effectuer le merge, on vérifie que la colonne 'Country' existe bien pour toutes les tables


### On commence par importer le module OS qui va nous permettre de travailler de manière globale sur les fichier plutôt que de les modifier un à un
import os


### On commence par supprimer des tables potentiellement en double
# On créer un variable correspondant au dossier contenant nos tables
dossier = "/home/onyxia/work/Projet_Python_2A/Data/EUDA/Cannabis détaillé/jamais_traite_entrants"


# On sélectionne les fichiers excel qui sont en double
fichiers1 = [f for f in os.listdir(dossier) if f.endswith('.xlsx')]


# On sépare en deux listes les fichier ayant et n'ayant pas de colonne nommée "Country"
Liste_1 = []
Liste_2 = []
for fichier in fichiers1:
    chemin_fichier = os.path.join(dossier, fichier)
    df = pd.read_excel(chemin_fichier, header=3)
    if "Country" in df.columns:  
        Liste_1.append(chemin_fichier)
    else :
        Liste_2.append(chemin_fichier)
        
print('nombre de tables dans la liste 1:',len(Liste_1), '\nnombre de tables dans la liste 2:',len(Liste_2))


nombre de tables dans la liste 1: 36 
nombre de tables dans la liste 2: 0


On a bien 36 tables dans la liste_1 et 0 dans Liste_2 : Nous sommes donc prêt pour effectuer le merge final.

#### 2.3.2 Etape préalable n°2 : supprimer les doublons (en matière de *Dataframe*)
On utilise le module **os** pour vérifier dans le titre des fichiers si apparaît le terme **"Copie"** qui permet donc, le cas échéant, de supprimer des fichiers en doublon.
L'existence de tels fichiers provient d'erreurs humaines ayant eu lieu lors du téléchargement des bases de données sur le site www.euda.europa.eu.

In [268]:
# Etape préalable : avant d'effectuer le merge, on supprime les tables en double (triple, etc)


### On commence par importer le module OS qui va nous permettre de travailler de manière globale sur les fichier plutôt que de les modifier un à un
import os


### On commence par supprimer des tables potentiellement en double
# On crée un dossier tampon qui va conserver nos tables modifiées
# On sélectionne les fichiers excel qui sont en double
fichiers1 = [f for f in os.listdir(dossier) if "Copie" in f and f.endswith('.xlsx')]
for fichier in fichiers1:
    chemin_fichier = os.path.join(dossier, fichier)
    if os.path.isfile(chemin_fichier):  # Vérifie si c'est bien un fichier
        os.remove(chemin_fichier)  # Supprime le fichier

#### 2.3.3 Etape préalable n°3 : supprimer la colonne Year of Treatment
Pourquoi un tel choix ? Cette colonne apparaît dans chaque table et donne des informations redondantes. Elle est, sauf très rares exceptions, identique dans chacune des *Dataframes*. Ainsi, afin d'éviter d'alourdir la table finale et pour faciliter le *Merge* (**problème avec les noms de colonnes redondants**), on fait le choix de supprimer de toutes les tables (**sauf pour table_base**) la colonne **"Year of Treatment"**.

In [269]:
warnings.filterwarnings('ignore')

### On va supprimer les colonnes "Year of Treatment" présentent dans toutes les tables pour éviter les doublons dans notre table finale
# A quelques exceptions prêt négligeables, ka date est toujours 2022, l'information est donc redondante
fichiers2 = [f for f in os.listdir(dossier) if f.endswith('.xlsx')]
# Chemin du nouveau dossier
chemin_dossier_nouveau = "/home/onyxia/work/Projet_Python_2A/Data/EUDA/Tampon"

# Créer le dossier s'il n'existe pas
os.makedirs(chemin_dossier_nouveau, exist_ok=True)

for fichier in fichiers2:
    chemin_fichier = os.path.join(dossier, fichier)
    chemin_fichier1 = os.path.join(chemin_dossier_nouveau, fichier)
    df = pd.read_excel(chemin_fichier, header=3)
    if "Year of Treatment" in df.columns :
        df = df.drop(columns=["Year of Treatment"])
    df.to_excel(chemin_fichier1, index=False) # On enregistre la modification, sinon le code ne sert à rien


#### 2.3.4 Etape préalable n°4 : 

Comme évoqué ci-dessus, il existe un problème en cas de trop forte **redondance dans le nom des colonnes** d'un *dataframe*.
Or le *Merge* de table ayant initialement des noms de colonnes identiques augment le risque d'être confronté à ce problème.

Ainsi, afin d'être prévoyant mais aussi de **faciliter la lecture de la table finale (*mergée)**, on rajoute des suffixes au nom de chaque colonne.

Ici, les suffixes concernent **les populations traitées dans le cadre des soins** et se décomposent entre :
- les personnes déjà traitées auparavant pour consommation de cannabis (**.deja**)
- les personnes jamais traitées auparavant pour consommation de cannabis (**.jamais**)
- l'ensemble des entrants dans un processus de soin pour sur une année et dans un pays donnés (**.global**)


Pour attribuer ces **suffixes**, on utilise le **module OS afin d'extraire cette information du nom du fichier**.

In [270]:
### On commence par ajouter le suffixe ".deja" aux colonnes des tables tables concernées
# On créer un variable correspondant au dossier contenant nos tables
dossier = "/home/onyxia/work/Projet_Python_2A/Data/EUDA/Tampon"


# On sélectionne les fichiers excel qui ont "deja" dans leur nom
fichiers1 = [f for f in os.listdir(dossier) if "deja" in f and f.endswith('.xlsx')]
fichiers = [f for f in os.listdir(dossier) if f.endswith('.xlsx')]


# On ajoute le suffixe aux colonnes souhaitées
for fichier in fichiers1:
    chemin_fichier = os.path.join(dossier, fichier)
    df = pd.read_excel(chemin_fichier)
    df.rename(columns={col: col + ".deja" if col not in ["Country", "Year of Treatment"] else col for col in df.columns}, inplace=True)
    df.to_excel(chemin_fichier, index=False) # On enregistre la modification, sinon le code ne sert à rien




### "jamais"
# On sélectionne les fichiers excel qui ont "jamais" dans leur nom
fichiers2 = [f for f in os.listdir(dossier) if "jamais" in f and f.endswith('.xlsx')]


# On ajoute le suffixe aux colonnes souhaitées
for fichier in fichiers2:
    chemin_fichier = os.path.join(dossier, fichier)
    df = pd.read_excel(chemin_fichier)
    df.rename(columns={col: col + ".jamais" if col not in ["Country", "Year of Treatment"] else col for col in df.columns}, inplace=True)
    df.to_excel(chemin_fichier, index=False) # On enregistre la modification, sinon le code ne sert à rien


### "global"
# On sélectionne les fichiers excel qui ont "global" dans leur nom
fichiers3 = [f for f in os.listdir(dossier) if "global" in f and f.endswith('.xlsx')]


# On ajoute le suffixe aux colonnes souhaitées
for fichier in fichiers3:
    chemin_fichier = os.path.join(dossier, fichier)
    df = pd.read_excel(chemin_fichier)
    df.rename(columns={col: col + ".global" if col not in ["Country", "Year of Treatment"] else col for col in df.columns}, inplace=True)
    df.to_excel(chemin_fichier, index=False) # On enregistre la modification, sinon le code ne sert à rien


test = "/home/onyxia/work/Projet_Python_2A/Data/EUDA/Tampon/age_traitement_global.xlsx"
test = pd.read_excel(test)




#### 2.3.5 Etape préalable 4bis :

Cette fois-ci, les **suffixes** concernent les **sous-catégories de populations** concernées par chaque **table** de l'EUDA.
Le **sujet de la table** est affichée avant chaque cellule code.

*Remarque : pour chaque sujet, il existe des **modalités** qui ne posent pas de souci en terme de redondance des noms de colonnes du fait de leur spécificité mais que nous traiterons plus tard dans un but de **lisibilité et de prise en main par des personnes non-familières avec les tables manipulées**.


**Statut pro**

*.pro*

In [271]:
import pandas as pd
import os
dossier_2 = "/home/onyxia/work/Projet_Python_2A/Data/EUDA/Tampon"
# On sélectionne les fichiers excel qui ont "pro" dans leur nom
fichiers1 = [f for f in os.listdir(dossier_2) if "pro" in f and f.endswith('.xlsx')]

# On ajoute le suffixe aux colonnes souhaitées
for fichier in fichiers1:
    chemin_fichier_2 = os.path.join(dossier_2, fichier) 
    df = pd.read_excel(chemin_fichier_2)
    df.columns = df.columns.astype(str)
    df = df.rename(columns={col: col + ".pro" if col != "Country" else col for col in df.columns})
    df.to_excel(chemin_fichier_2, index=False) # On enregistre la modification, sinon le code ne sert à rien



**sexe**

*.sex*

In [272]:
# On sélectionne les fichiers excel qui ont "sex" dans leur nom
fichiers2 = [f for f in os.listdir(dossier_2) if "sex" in f and f.endswith('.xlsx')]

# On ajoute le suffixe aux colonnes souhaitées
for fichier in fichiers2:
    chemin_fichier_2 = os.path.join(dossier_2, fichier) 
    df = pd.read_excel(chemin_fichier_2)
    df.columns = df.columns.astype(str)
    df = df.rename(columns={col: col + ".sex" if col != "Country" else col for col in df.columns})
    df.to_excel(chemin_fichier_2, index=False) # On enregistre la modification, sinon le code ne sert à rien


**sdf**

*.sdf*

In [273]:
# On sélectionne les fichiers excel qui ont "sex" dans leur nom
fichiers3 = [f for f in os.listdir(dossier_2) if "sdf" in f and f.endswith('.xlsx')]

# On ajoute le suffixe aux colonnes souhaitées
for fichier in fichiers3:
    chemin_fichier_2 = os.path.join(dossier_2, fichier) 
    df = pd.read_excel(chemin_fichier_2)
    df.columns = df.columns.astype(str)
    df = df.rename(columns={col: col + ".sdf" if col != "Country" else col for col in df.columns})
    df.to_excel(chemin_fichier_2, index=False) # On enregistre la modification, sinon le code ne sert à rien

**Niveau d'éducation**

*.lvl_educ*

In [274]:
# On sélectionne les fichiers excel qui ont "niveau" dans leur nom
fichiers4 = [f for f in os.listdir(dossier_2) if "niveau" in f and f.endswith('.xlsx')]

# On ajoute le suffixe aux colonnes souhaitées
for fichier in fichiers4:
    chemin_fichier_2 = os.path.join(dossier_2, fichier) 
    df = pd.read_excel(chemin_fichier_2)
    df.columns = df.columns.astype(str)
    df = df.rename(columns={col: col + ".lvl_educ" if col != "Country" else col for col in df.columns})
    df.to_excel(chemin_fichier_2, index=False) # On enregistre la modification, sinon le code ne sert à rien

**Fréquence de consommation**

*.frqce_conso*

In [275]:
# On sélectionne les fichiers excel qui ont "frequence" dans leur nom
fichiers5 = [f for f in os.listdir(dossier_2) if "frequence" in f and f.endswith('.xlsx')]

# On ajoute le suffixe aux colonnes souhaitées
for fichier in fichiers5:
    chemin_fichier_2 = os.path.join(dossier_2, fichier) 
    df = pd.read_excel(chemin_fichier_2)
    df.columns = df.columns.astype(str)
    df = df.rename(columns={col: col + ".frqce_conso" if col != "Country" else col for col in df.columns})
    df.to_excel(chemin_fichier_2, index=False) # On enregistre la modification, sinon le code ne sert à rien

**Nombre d'enfants**

*.enfants*

In [276]:
# On sélectionne les fichiers excel qui ont "enfants" dans leur nom
fichiers6 = [f for f in os.listdir(dossier_2) if "enfants" in f and f.endswith('.xlsx')]

# On ajoute le suffixe aux colonnes souhaitées
for fichier in fichiers6:
    chemin_fichier_2 = os.path.join(dossier_2, fichier) 
    df = pd.read_excel(chemin_fichier_2)
    df.columns = df.columns.astype(str)
    df = df.rename(columns={col: col + ".enfants" if col != "Country" else col for col in df.columns})
    df.to_excel(chemin_fichier_2, index=False) # On enregistre la modification, sinon le code ne sert à rien

**Composition du ménage**

*.menage*

In [277]:
# On sélectionne les fichiers excel qui ont "menage" dans leur nom
fichiers7 = [f for f in os.listdir(dossier_2) if "menage" in f and f.endswith('.xlsx')]

# On ajoute le suffixe aux colonnes souhaitées
for fichier in fichiers7:
    chemin_fichier_2 = os.path.join(dossier_2, fichier) 
    df = pd.read_excel(chemin_fichier_2)
    df.columns = df.columns.astype(str)
    df = df.rename(columns={col: col + ".menage" if col != "Country" else col for col in df.columns})
    df.to_excel(chemin_fichier_2, index=False) # On enregistre la modification, sinon le code ne sert à rien

**Age de l'entrant**

*.age_trait*

In [278]:
# On sélectionne les fichiers excel qui ont "traitement" dans leur nom
fichiers8 = [f for f in os.listdir(dossier_2) if "traitement" in f and f.endswith('.xlsx')]

# On ajoute le suffixe aux colonnes souhaitées
for fichier in fichiers8:
    chemin_fichier_2 = os.path.join(dossier_2, fichier) 
    df = pd.read_excel(chemin_fichier_2)
    df.columns = df.columns.astype(str)
    df = df.rename(columns={col: col + ".age_trait" if col != "Country" else col for col in df.columns})
    df.to_excel(chemin_fichier_2, index=False) # On enregistre la modification, sinon le code ne sert à rien

**Age lors de la première consommation**

*.age_premiere_conso*

In [279]:
# On sélectionne les fichiers excel qui ont "premier" dans leur nom
fichiers9 = [f for f in os.listdir(dossier_2) if "premier" in f and f.endswith('.xlsx')]

# On ajoute le suffixe aux colonnes souhaitées
for fichier in fichiers9:
    chemin_fichier_2 = os.path.join(dossier_2, fichier) 
    df = pd.read_excel(chemin_fichier_2)
    df.columns = df.columns.astype(str)
    df = df.rename(columns={col: col + ".age_premiere_conso" if col != "Country" else col for col in df.columns})
    df.to_excel(chemin_fichier_2, index=False) # On enregistre la modification, sinon le code ne sert à rien

**De qui a émané la demande de soins**
(institution judiciaire, proches, personne concernée, *etc*)

*.demande*

In [280]:
# On sélectionne les fichiers excel qui ont "demande" dans leur nom
fichiers10 = [f for f in os.listdir(dossier_2) if "demande" in f and f.endswith('.xlsx')]

# On ajoute le suffixe aux colonnes souhaitées
for fichier in fichiers10:
    chemin_fichier_2 = os.path.join(dossier_2, fichier) 
    df = pd.read_excel(chemin_fichier_2)
    df.columns = df.columns.astype(str)
    df = df.rename(columns={col: col + ".demande" if col != "Country" else col for col in df.columns})
    df.to_excel(chemin_fichier_2, index=False) # On enregistre la modification, sinon le code ne sert à rien

### 2.4 On effectue enfin le merge final

Maintenant que les principaux soucis (noms de colonnes, présence de la colonne 'Country' dans chaque *Dataframe*) sont réglés, on peut passer à l'étape tant attendue : la création d'une **table récapitulative sur les données liées aux entrants dans des processus de soins pour des raisons de consommation de cannabis**.

Pour procéder au *Merge*, on va chercher chaque fichier dans des **listes** établies pour attribuer les suffixes aux noms des colonnes des *tables* et ainsi optimiser le code en ne multipliant pas de trop les lignes redondantes, mais en gardant tout de même une subdivision utile pour identifier un potentiel problème lors du *Merge*.

In [ ]:
### Maintenant que ces modifications sont faites, on effectue le merge
# On effectue autant de merge que nécessaire en créant de nouveaux suffixes à chaque fois
test=table_base
dossier = "/home/onyxia/work/Projet_Python_2A/Data/EUDA/Tampon"
for fichier in fichiers1 :
    chemin_fichier = os.path.join(dossier, fichier)
    df = pd.read_excel(chemin_fichier)
    test = pd.merge(test, df, how="left", on="Country")
for fichier in fichiers2 :
    chemin_fichier = os.path.join(dossier, fichier)
    df = pd.read_excel(chemin_fichier)
    test = pd.merge(test, df, how="left", on="Country")
for fichier in fichiers3 :
    chemin_fichier = os.path.join(dossier, fichier)
    df = pd.read_excel(chemin_fichier)
    test = pd.merge(test, df, how="left", on="Country")
for fichier in fichiers4 :
    chemin_fichier = os.path.join(dossier, fichier)
    df = pd.read_excel(chemin_fichier)
    test = pd.merge(test, df, how="left", on="Country")
for fichier in fichiers5 :
    chemin_fichier = os.path.join(dossier, fichier)
    df = pd.read_excel(chemin_fichier)
    test = pd.merge(test, df, how="left", on="Country")
for fichier in fichiers6 :
    chemin_fichier = os.path.join(dossier, fichier)
    df = pd.read_excel(chemin_fichier)
    test = pd.merge(test, df, how="left", on="Country")
for fichier in fichiers7 :
    chemin_fichier = os.path.join(dossier, fichier)
    df = pd.read_excel(chemin_fichier)
    test = pd.merge(test, df, how="left", on="Country")
for fichier in fichiers8 :
    chemin_fichier = os.path.join(dossier, fichier)
    df = pd.read_excel(chemin_fichier)
    test = pd.merge(test, df, how="left", on="Country")
for fichier in fichiers9 :
    chemin_fichier = os.path.join(dossier, fichier)
    df = pd.read_excel(chemin_fichier)
    test = pd.merge(test, df, how="left", on="Country")
for fichier in fichiers10 :
    chemin_fichier = os.path.join(dossier, fichier)
    df = pd.read_excel(chemin_fichier)
    test = pd.merge(test, df, how="left", on="Country")
soins_cannabis = test

soins_cannabis.head() #On vérifie que le résultat est bien celui souhaité

,Country,Year of Treatment,All substances (global),Cannabis (global),All substances (deja traite.es),Cannabis (deja traite.es),All substances (jamais traite.es),Cannabis (jamais traite.es),2022.0.plante.conso,2021.0.plante.conso,...,Total.global.demande,Court / probation / police.jamais.demande,General practitioner.jamais.demande,Other drug treatment centre.jamais.demande,"Other health, medical or social service.jamais.demande",Educational services.jamais.demande,"Self-referral, referral from family, friends, etc.; no other agency/institution involved.jamais.demande",Other.jamais.demande,Not known / missing.jamais.demande,Total.jamais.demande
0,Austria,2022.0,4709.0,1124.0,2756.0,458.0,1953.0,666.0,617.0,717.0,...,1124.0,179.0,15.0,11.0,190.0,16.0,209.0,39.0,7.0,666.0
1,Belgium,2022.0,12144.0,3501.0,7376.0,1524.0,4151.0,1850.0,54.0,51.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Bulgaria,2022.0,1879.0,109.0,1164.0,42.0,605.0,66.0,NaN,NaN,...,109.0,12.0,0.0,2.0,2.0,0.0,41.0,9.0,0.0,66.0
3,Croatia *,2022.0,2315.0,478.0,NaN,NaN,NaN,NaN,NaN,NaN,...,478.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Cyprus,2022.0,884.0,340.0,308.0,66.0,535.0,251.0,NaN,NaN,...,340.0,124.0,0.0,6.0,7.0,1.0,62.0,48.0,3.0,251.0


## 3. Ajout de nouvelles tables : infractions à la loi

Ici, on va procéder de manière analogue à la partie précédente sur l'ajout des tables en matière de soins et de traitements liés à la consommation de drogues en Europe en l'appliquant cette fois à des données sur les **infractions légales constatées liées au cannabis (vente, production, consommation)**.

### 3.1 Importation des tables
On utilise les modules **Pandas** et **OS** afin de passer les fichiers du format .xslx (Excel) au format Pandas.

#### 3.1.1 Vérification du type de fichier
On a rencontré au cours de notre travail des problèmes à causes de tables téléchargées au mauvais format, ce qui empêchait le code de tourner correctement.
Pour éviter de rencontre à nouveau ce type de souci, on vérifie que tous les fichiers sont bien de type *.xlsx*.

*Remarque : Cette étape à ce stade inutile (le problème ayant déjà été réglé une fois) permet, au-delà de la reproductibilité du code, de lui permettre de s'adapter à d'autres bases de données téléchargées manuellement et qui pourraient contenir de nouvelles erreurs de types de fichier par encore corrigées.*

In [282]:
import os
liste = []
### On crée une fonction pour récupérer le type d'un fichier basé sur son extension grâce au module os
def get_file_type(file_path):
    _, file_extension = os.path.splitext(file_path)
    return file_extension.lower()  # On met en minuscule pour éviter les problèmes



### On ajoute le type de chaque fichier à une liste
fichiers7 = [f for f in os.listdir(dossier) if f.endswith('.xlsx')]

# On ajoute le suffixe aux colonnes souhaitées
for fichier in fichiers7:
    liste.append(get_file_type(fichier))

### On fait un test pour vérifier que tous les fichiers sont bien de .xlsx
all_equal = all(x == liste[0] for x in liste)
print(all_equal)

True


**Tous les fichiers sont bien des *.xlsx* désormais**

### 3.2 On spécifie le type de drogue concernée

L'idée est ici de faire en sorte de rendre les tables plus **lisibles**, en particulier en vue d'un **merge** de celles-ci.
Le nom des colonnes étant redondant et peu lisible sans le nom de la table (2010, 2011, *etc*), l'idée est de donner via les **noms  de colonnes** des **clés de lecture**.

Pour cela, nous allons ajouter des informations au nom de colonne *via* le format suivant :
**année.détail1.détail2**



Voici la classification choisie :

**Détail1** :
Regroupe les différents types de drogues étudiées (glbal (*i.e.* toutes les drogues), cannabis (*i.e.* sous toutes ses formes), herbe, raisine, huile et plante).

**Détail2**
Regroupe les différents types d'infractions (vente, consommation, production, infractions/délinquants (total des différents types d'infractions)).

**Cas global**

*.cannabis*

In [283]:
import os
import pandas as pd

warnings.filterwarnings('ignore')

### On commence par ajouter le suffixe ".cannabis" aux colonnes des tables concernées
# On créer un variable correspondant au dossier contenant nos tables
dossier = "/home/onyxia/work/Projet_Python_2A/Data/EUDA/Delits_Crimes"
dossier_2 = "/home/onyxia/work/Projet_Python_2A/Data/EUDA/Tampon"
fichier = [f for f in os.listdir(dossier) if f.endswith('.xlsx')]

# On sélectionne les fichiers excel qui ont "Cannabis" dans leur nom
fichiers1 = [f for f in os.listdir(dossier) if "Cannabis" in f and f.endswith('.xlsx')]

# On ajoute le suffixe aux colonnes souhaitées
for fichier in fichiers1:
    chemin_fichier = os.path.join(dossier, fichier)
    chemin_fichier_2 = os.path.join(dossier_2, fichier) 
    df = pd.read_excel(chemin_fichier)
# On utilise la ligne 3 (index 2) comme nouveaux titres de colonnes
    df.columns = df.iloc[2]  # Utiliser la troisième ligne comme titres 
# On supprime la ligne qui est devenue les titres de colonnes
    df = df.drop(index=2)
# On réinitialise les index après la suppression
    df.reset_index(drop=True, inplace=True)
    df.columns = df.columns.astype(str)
    df = df.rename(columns={col: col + ".cannabis" if col != "Country" else col for col in df.columns})
    df.to_excel(chemin_fichier_2, index=False) # On enregistre la modification, sinon le code ne sert à rien


**Herbe**

*.herbe*

In [284]:
warnings.filterwarnings('ignore')

### "Herbe"
# On sélectionne les fichiers excel qui ont "Herbe" dans leur nom
fichiers2 = [f for f in os.listdir(dossier) if "Herbe" in f and f.endswith('.xlsx')]

# On ajoute le suffixe aux colonnes souhaitées
for fichier in fichiers2:
    chemin_fichier = os.path.join(dossier, fichier)
    chemin_fichier_2 = os.path.join(dossier_2, fichier) 
    df = pd.read_excel(chemin_fichier)
# On utilise la ligne 3 (index 2) comme nouveaux titres de colonnes
    df.columns = df.iloc[2]  # Utiliser la troisième ligne comme titres 
# On supprime la ligne qui est devenue les titres de colonnes
    df = df.drop(index=2)
# On réinitialise les index après la suppression
    df.reset_index(drop=True, inplace=True)
    df.columns = df.columns.astype(str)
    df = df.rename(columns={col: col + ".herbe" if col != "Country" else col for col in df.columns})
    df.to_excel(chemin_fichier_2, index=False) # On enregistre la modification, sinon le code ne sert à rien



**Huile**

*.huile*

In [285]:
warnings.filterwarnings('ignore')

### "Huile"
# On sélectionne les fichiers excel qui ont "Huile" dans leur nom
fichiers3 = [f for f in os.listdir(dossier) if "Huile" in f and f.endswith('.xlsx')]

# On ajoute le suffixe aux colonnes souhaitées
for fichier in fichiers3:
    chemin_fichier = os.path.join(dossier, fichier)
    chemin_fichier_2 = os.path.join(dossier_2, fichier) 
    df = pd.read_excel(chemin_fichier)
# On utilise la ligne 3 (index 2) comme nouveaux titres de colonnes
    df.columns = df.iloc[2]  # Utiliser la troisième ligne comme titres 
# On supprime la ligne qui est devenue les titres de colonnes
    df = df.drop(index=2)
# On réinitialise les index après la suppression
    df.reset_index(drop=True, inplace=True)
    df.columns = df.columns.astype(str)
    df = df.rename(columns={col: col + ".huile" if col != "Country" else col for col in df.columns})
    df.to_excel(chemin_fichier_2, index=False) # On enregistre la modification, sinon le code ne sert à rien


**Plante**

*.plante*

In [286]:
warnings.filterwarnings('ignore')

### "Plante"
# On sélectionne les fichiers excel qui ont "Plante" dans leur nom
fichiers5 = [f for f in os.listdir(dossier) if "Plante" in f and f.endswith('.xlsx')]


# On ajoute le suffixe aux colonnes souhaitées
for fichier in fichiers5:
    chemin_fichier = os.path.join(dossier, fichier)
    chemin_fichier_2 = os.path.join(dossier_2, fichier) 
    df = pd.read_excel(chemin_fichier)
# On utilise la ligne 3 (index 2) comme nouveaux titres de colonnes
    df.columns = df.iloc[2]  # Utiliser la troisième ligne comme titres 
# On supprime la ligne qui est devenue les titres de colonnes
    df = df.drop(index=2)
# On réinitialise les index après la suppression
    df.reset_index(drop=True, inplace=True)
    df.columns = df.columns.astype(str)
    df = df.rename(columns={col: col + ".plante" if col != "Country" else col for col in df.columns})
    df.to_excel(chemin_fichier_2, index=False) # On enregistre la modification, sinon le code ne sert à rien


**Raisine**

*.raisine*

In [287]:
warnings.filterwarnings('ignore')

### "Raisine"
# On sélectionne les fichiers excel qui ont "Raisine" dans leur nom
fichiers6 = [f for f in os.listdir(dossier) if "Raisine" in f and f.endswith('.xlsx')]

# On ajoute le suffixe aux colonnes souhaitées
for fichier in fichiers6:
    chemin_fichier = os.path.join(dossier, fichier)
    chemin_fichier_2 = os.path.join(dossier_2, fichier) 
    df = pd.read_excel(chemin_fichier)
# On utilise la ligne 3 (index 2) comme nouveaux titres de colonnes
    df.columns = df.iloc[2]  # Utiliser la troisième ligne comme titres 
# On supprime la ligne qui est devenue les titres de colonnes
    df = df.drop(index=2)
# On réinitialise les index après la suppression
    df.reset_index(drop=True, inplace=True)
    df.columns = df.columns.astype(str)
    df = df.rename(columns={col: col + ".raisine" if col != "Country" else col for col in df.columns})
    df.to_excel(chemin_fichier_2, index=False) # On enregistre la modification, sinon le code ne sert à rien


**Global**

(toutes les drogues, dont cannabis)

*.global*

In [288]:
warnings.filterwarnings('ignore')

### "Global"
# On sélectionne les fichiers excel qui ont "global" dans leur nom
fichiers7 = [f for f in os.listdir(dossier) if "Global" in f and f.endswith('.xlsx')]

# On ajoute le suffixe aux colonnes souhaitées
for fichier in fichiers7:
    chemin_fichier = os.path.join(dossier, fichier)
    chemin_fichier_2 = os.path.join(dossier_2, fichier) 
    df = pd.read_excel(chemin_fichier)
# On utilise la ligne 3 (index 2) comme nouveaux titres de colonnes
    df.columns = df.iloc[2]  # Utiliser la troisième ligne comme titres 
# On supprime la ligne qui est devenue les titres de colonnes
    df = df.drop(index=2)
# On réinitialise les index après la suppression
    df.reset_index(drop=True, inplace=True)
    df.columns = df.columns.astype(str)
    df = df.rename(columns={col: col + ".global" if col != "Country" else col for col in df.columns})
    df.to_excel(chemin_fichier_2, index=False) # On enregistre la modification, sinon le code ne sert à rien


### 3.3 On spécifie le type d'infractions observées

Cette fois-ci (toujours avec le même *modus operandi*), on rajoute des **suffixes en fonction du type d'infraction constatée**.

**Consommation**

*.conso*

In [289]:
# On sélectionne les fichiers excel qui ont "Consommation" dans leur nom
fichiers1 = [f for f in os.listdir(dossier) if "Consommation" in f and f.endswith('.xlsx')]

# On ajoute le suffixe aux colonnes souhaitées
for fichier in fichiers1:
    chemin_fichier_2 = os.path.join(dossier_2, fichier) 
    df = pd.read_excel(chemin_fichier_2)
    df.columns = df.columns.astype(str)
    # On sélectionne les lignes de l'index 2 à 31 (inclus)
    df = df.iloc[2:32, :]
    df = df.rename(columns={col: col + ".conso" if col != "Country" else col for col in df.columns})
    df.to_excel(chemin_fichier_2, index=False) # On enregistre la modification, sinon le code ne sert à rien

**Délinquants**

(s'applique uniquement pour une colonne concernant les drogues dans leur ensemble)

*.delinquants* 

In [290]:
### "Delinquants"
# On sélectionne les fichiers excel qui ont "Delinquants" dans leur nom
fichiers2 = [f for f in os.listdir(dossier) if "Deliquants" in f and f.endswith('.xlsx')]


# On ajoute le suffixe aux colonnes souhaitées
for fichier in fichiers2:
    chemin_fichier_2 = os.path.join(dossier_2, fichier) 
    df = pd.read_excel(chemin_fichier_2)
    df.columns = df.columns.astype(str)
    # On sélectionne les lignes de l'index 2 à 31 (inclus)
    df = df.iloc[2:32, :]
    df = df.rename(columns={col: col + ".delinquants" if col != "Country" else col for col in df.columns})
    df.to_excel(chemin_fichier_2, index=False) # On enregistre la modification, sinon le code ne sert à rien

**Infractions**

*.infractions*

In [291]:
### "Infractions"
# On sélectionne les fichiers excel qui ont "Infractions" dans leur nom
fichiers3 = [f for f in os.listdir(dossier) if "Infractions" in f and f.endswith('.xlsx')]


# On ajoute le suffixe aux colonnes souhaitées
for fichier in fichiers3:
    chemin_fichier_2 = os.path.join(dossier_2, fichier) 
    df = pd.read_excel(chemin_fichier_2)
    df.columns = df.columns.astype(str)
    # On sélectionne les lignes de l'index 2 à 31 (inclus)
    df = df.iloc[2:32, :]
    df = df.rename(columns={col: col + ".infractions" if col != "Country" else col for col in df.columns})
    df.to_excel(chemin_fichier_2, index=False) # On enregistre la modification, sinon le code ne sert à rien

**Production**

*.prod*

In [292]:
### "Production"
# On sélectionne les fichiers excel qui ont "Production" dans leur nom
fichiers4 = [f for f in os.listdir(dossier) if "Production" in f and f.endswith('.xlsx')]


# On ajoute le suffixe aux colonnes souhaitées
for fichier in fichiers4:
    chemin_fichier_2 = os.path.join(dossier_2, fichier) 
    df = pd.read_excel(chemin_fichier_2)
    df.columns = df.columns.astype(str)
    # On sélectionne les lignes de l'index 2 à 31 (inclus)
    df = df.iloc[2:32, :]
    df = df.rename(columns={col: col + ".prod" if col != "Country" else col for col in df.columns})
    df.to_excel(chemin_fichier_2, index=False) # On enregistre la modification, sinon le code ne sert à rien

**Ventes**

*.ventes*

In [293]:
### "Ventes"
# On sélectionne les fichiers excel qui ont "Ventes" dans leur nom
fichiers5 = [f for f in os.listdir(dossier) if "Ventes" in f and f.endswith('.xlsx')]


# On ajoute le suffixe aux colonnes souhaitées
for fichier in fichiers5:
    chemin_fichier_2 = os.path.join(dossier_2, fichier) 
    df = pd.read_excel(chemin_fichier_2)
    df.columns = df.columns.astype(str)
    # On sélectionne les lignes de l'index 2 à 31 (inclus)
    df = df.iloc[2:32, :]
    df = df.rename(columns={col: col + ".ventes" if col != "Country" else col for col in df.columns})
    df.to_excel(chemin_fichier_2, index=False) # On enregistre la modification, sinon le code ne sert à rien

### 3.4 On passe au Merge final

On effectue un merge sur toutes les tables concernant les infractions à la loi ayant trait à la drogue (consommation, production, etc) maintenant que les modifications nécessaires à la **lisibilité** et au **bon fonctionnement du code** ont été faites.
Le fait d'avoir modifier le nom des colonnes permet en effet de donner un résultat plus lisible.

In [ ]:
### Maintenant que ces modifications sont faites, on effectue le merge
# On effectue autant de merge que nécessaire en créant de nouveaux suffixes à chaque fois
test= pd.read_excel("/home/onyxia/work/Projet_Python_2A/Data/EUDA/Tampon/Nombre_Consommation_Cannabis.xlsx")
dossier = "/home/onyxia/work/Projet_Python_2A/Data/EUDA/Tampon"
for fichier in fichiers1 :
    chemin_fichier = os.path.join(dossier, fichier)
    df = pd.read_excel(chemin_fichier)
    test = pd.merge(test, df, how="left", on="Country")
for fichier in fichiers2 :
    chemin_fichier = os.path.join(dossier, fichier)
    df = pd.read_excel(chemin_fichier)
    test = pd.merge(test, df, how="left", on="Country")
for fichier in fichiers3 :
    chemin_fichier = os.path.join(dossier, fichier)
    df = pd.read_excel(chemin_fichier)
    test = pd.merge(test, df, how="left", on="Country")
for fichier in fichiers4 :
    chemin_fichier = os.path.join(dossier, fichier)
    df = pd.read_excel(chemin_fichier)
    test = pd.merge(test, df, how="left", on="Country")
for fichier in fichiers5 :
    chemin_fichier = os.path.join(dossier, fichier)
    df = pd.read_excel(chemin_fichier)
    test = pd.merge(test, df, how="left", on="Country")
table_infractions_legales = test

table_infractions_legales.head() #On vérifie le résultat

,Country,2022.0.cannabis.conso_x,2021.0.cannabis.conso_x,2020.0.cannabis.conso_x,2019.0.cannabis.conso_x,2018.0.cannabis.conso_x,2017.0.cannabis.conso_x,2016.0.cannabis.conso_x,2015.0.cannabis.conso_x,2014.0.cannabis.conso_x,...,2016.0.cannabis.ventes,2015.0.cannabis.ventes,2014.0.cannabis.ventes,2013.0.cannabis.ventes,2012.0.cannabis.ventes,2011.0.cannabis.ventes,2010.0.cannabis.ventes,2009.0.cannabis.ventes,2008.0.cannabis.ventes,2007.0.cannabis.ventes
0,Austria,19909.0,19919.0,22778.0,24618.0,24680.0,28890.0,23977.0,20456.0,21272.0,...,7372.0,7086.0,6705.0,1595.0,1409.0,1428.0,1454.0,1460.0,1261.0,1833.0
1,Belgium,29572.0,28857.0,31540.0,31688.0,30148.0,29916.0,27080.0,28237.0,29825.0,...,6316.0,6460.0,6637.0,6074.0,5942.0,8341.0,8574.0,8892.0,9047.0,8200.0
2,Bulgaria,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3465.0,...,NaN,NaN,790.0,1424.0,800.0,625.0,627.0,641.0,438.0,314.0
3,Croatia,5159.0,5080.0,4810.0,5689.0,6788.0,6253.0,5493.0,4268.0,4866.0,...,1719.0,1620.0,1399.0,1799.0,1442.0,1790.0,1746.0,1519.0,931.0,1199.0
4,Cyprus,563.0,512.0,606.0,692.0,862.0,703.0,643.0,690.0,799.0,...,106.0,142.0,88.0,127.0,124.0,88.0,117.0,129.0,NaN,128.0


## 4. Création de tables simplifiées
On crée deux tables simplifiées ne gardant que les informations sur le **cannabis** et les **drogues en général** et plus sur les **différents types de cannabis** (raisine, herbe, *etc*) ni sur les différents profils  d'**entrants en matière de passif de traitements (on ne garde que le total des entrants par pays sur l'année concernée)**.

On fait ça pour les deux tables "finales" **table_infractions_legales** et pour **soins_cannabis**.

### 4.1 Pour la table concernant les infractions légales
On se sert des spécifications dans le **nom des colonnes *.detail1***.
Ainsi, on va donc uniquement sélectionner les colonnes contenant "**.cannabis**" et "**.global**".

In [311]:
# Sélection des colonnes contenant ".cannabis" et ".global"
table_infractions_simple = table_infractions_legales[[col for col in table_infractions_legales.columns if ".cannabis" in col or ".global" in col or col == "Country"]]

table_infractions_simple.head()

,Country,2022.0.cannabis.conso_x,2021.0.cannabis.conso_x,2020.0.cannabis.conso_x,2019.0.cannabis.conso_x,2018.0.cannabis.conso_x,2017.0.cannabis.conso_x,2016.0.cannabis.conso_x,2015.0.cannabis.conso_x,2014.0.cannabis.conso_x,...,2016.0.cannabis.ventes,2015.0.cannabis.ventes,2014.0.cannabis.ventes,2013.0.cannabis.ventes,2012.0.cannabis.ventes,2011.0.cannabis.ventes,2010.0.cannabis.ventes,2009.0.cannabis.ventes,2008.0.cannabis.ventes,2007.0.cannabis.ventes
0,Austria,19909.0,19919.0,22778.0,24618.0,24680.0,28890.0,23977.0,20456.0,21272.0,...,7372.0,7086.0,6705.0,1595.0,1409.0,1428.0,1454.0,1460.0,1261.0,1833.0
1,Belgium,29572.0,28857.0,31540.0,31688.0,30148.0,29916.0,27080.0,28237.0,29825.0,...,6316.0,6460.0,6637.0,6074.0,5942.0,8341.0,8574.0,8892.0,9047.0,8200.0
2,Bulgaria,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3465.0,...,NaN,NaN,790.0,1424.0,800.0,625.0,627.0,641.0,438.0,314.0
3,Croatia,5159.0,5080.0,4810.0,5689.0,6788.0,6253.0,5493.0,4268.0,4866.0,...,1719.0,1620.0,1399.0,1799.0,1442.0,1790.0,1746.0,1519.0,931.0,1199.0
4,Cyprus,563.0,512.0,606.0,692.0,862.0,703.0,643.0,690.0,799.0,...,106.0,142.0,88.0,127.0,124.0,88.0,117.0,129.0,NaN,128.0


On est passé de **417** colonnes à **160 colonnes**.

### 4.2 Pour la table concernant les soins

On se sert des spécifications dans le **nom des colonnes *.detail***.
Ainsi, on va donc uniquement sélectionner les colonnes contenant "**global**. Pas juste **".global**" pour ne pas exclure les colonnes "All Substances (global)" et "Cannabis (global)", mais afin de seulement exclure *.jamais* et *.deja*.

In [197]:
# Sélection des colonnes contenant ".global"
table_soins_simple = soins_cannabis[[col for col in soins_cannabis.columns if "global" in col or col == "Country" or col == "Year of Treatment"]]

# On affiche le résultat pour vérifier que cela a correctment fonctionné
table_soins_simple.head()

,Country,Year of Treatment,All substances (global),Cannabis (global),Occasionally employed.global.pro,Regularly employed.global.pro,Student.global.pro,Unemployed / discouraged.global.pro,Receiving social benefits / pensioners / house-makers / disable.global.pro,Other.global.pro,...,Total.global.age_premiere_conso,Court / probation / police.global.demande,General practitioner.global.demande,Other drug treatment centre.global.demande,"Other health, medical or social service.global.demande",Educational services.global.demande,"Self-referral, referral from family, friends, etc.; no other agency/institution involved.global.demande",Other.global.demande,Not known / missing.global.demande,Total.global.demande
0,Austria,2022.0,4709.0,1124.0,13.0,440.0,75.0,287.0,40.0,17.0,...,1124.0,337.0,23.0,30.0,275.0,19.0,353.0,54.0,33.0,1124.0
1,Belgium,2022.0,12144.0,3501.0,142.0,949.0,776.0,652.0,723.0,50.0,...,3501.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Bulgaria,2022.0,1879.0,109.0,4.0,19.0,21.0,53.0,11.0,1.0,...,109.0,28.0,1.0,4.0,3.0,0.0,63.0,9.0,1.0,109.0
3,Croatia *,2022.0,2315.0,478.0,53.0,102.0,145.0,135.0,7.0,8.0,...,478.0,119.0,20.0,0.0,95.0,NaN,197.0,32.0,15.0,478.0
4,Cyprus,2022.0,884.0,340.0,17.0,116.0,77.0,98.0,5.0,16.0,...,340.0,139.0,0.0,12.0,10.0,1.0,104.0,68.0,6.0,340.0


On est passé de **257** colonnes à **87 colonnes**.

## 5. Valeurs manquantes : colonne par colonne

### 5.1 Comptage des valeurs manquantes

On crée une **ligne** supplémentaire dans chaque *dataframe* qui affichera pour **chaque colonne** la valeur prise par un **fonction** qui compte les **valeurs manquantes**.

**Pourquoi procéder ainsi ?**
L'étape suivante consistera à choisir les variables (colonnes ou groupe de colonnes) pour lesquelles on a le moins de valeurs manquantes.

**NB : on en profite aussi pour convertir toutes les colonnes - sauf "Country" - en NOMBRE ENTIER afin de rendre la table plus lisible (en effet, les données sont soit des dates, soit des nombres de patients)**

#### 5.1.1 On commence par table_soins_simple

Tout d'abord, on s'occupe d'**uniformiser les valeurs manquantes et aberrantes** sous la forme "NaN".
Ainsi, on agit sur toutes les colonnes pour que "**inf**" et "**-inf**" s'affiche également comme NaN.

Ensuite, on fait en sorte que toutes les valeurs soient des **entiers (Int)** pour les raisons décrites précédemment.

In [297]:
warnings.filterwarnings('ignore')

for col in table_soins_simple.columns:
    if col != 'Country':  # On exclut la colonne 'Country'
        table_soins_simple[col] = pd.to_numeric(table_soins_simple[col], errors='coerce')  # Coerce transforme les valeurs non convertibles en NaN
        table_soins_simple[col] = table_soins_simple[col].astype('Int64')
        
# On affiche le dataframe pour vérifier que l'opération a fonctionné correctement
table_soins_simple.head()

,Country,Year of Treatment,All substances (global),Cannabis (global),Occasionally employed.global.pro,Regularly employed.global.pro,Student.global.pro,Unemployed / discouraged.global.pro,Receiving social benefits / pensioners / house-makers / disable.global.pro,Other.global.pro,...,Total.global.age_premiere_conso,Court / probation / police.global.demande,General practitioner.global.demande,Other drug treatment centre.global.demande,"Other health, medical or social service.global.demande",Educational services.global.demande,"Self-referral, referral from family, friends, etc.; no other agency/institution involved.global.demande",Other.global.demande,Not known / missing.global.demande,Total.global.demande
0,Austria,2022,4709,1124,13,440,75,287,40,17,...,1124,337,23,30,275,19,353,54,33,1124
1,Belgium,2022,12144,3501,142,949,776,652,723,50,...,3501,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>
2,Bulgaria,2022,1879,109,4,19,21,53,11,1,...,109,28,1,4,3,0,63,9,1,109
3,Croatia *,2022,2315,478,53,102,145,135,7,8,...,478,119,20,0,95,<NA>,197,32,15,478
4,Cyprus,2022,884,340,17,116,77,98,5,16,...,340,139,0,12,10,1,104,68,6,340


Ceci étant fait, on crée une **fonction de comptage des valeurs manquantes** qui s'affichera comme une nouvelle ligne pour chaque colonne (sauf "Country" et "Year of Treatment").

In [298]:
warnings.filterwarnings('ignore')

# On compte le nombre de NaN dans chaque colonne du dataframe "table_soins_simple"
Nb_valeurs_manquantes = table_soins_simple.isna().sum()

# On ajoute une ligne contenant cette variable au dataframe
table_soins_simple.loc['Nb_valeurs_manquantes'] = Nb_valeurs_manquantes

# On affiche le dataframe pour vérifier que l'opération a fonctionné correctement
table_soins_simple.head()

,Country,Year of Treatment,All substances (global),Cannabis (global),Occasionally employed.global.pro,Regularly employed.global.pro,Student.global.pro,Unemployed / discouraged.global.pro,Receiving social benefits / pensioners / house-makers / disable.global.pro,Other.global.pro,...,Total.global.age_premiere_conso,Court / probation / police.global.demande,General practitioner.global.demande,Other drug treatment centre.global.demande,"Other health, medical or social service.global.demande",Educational services.global.demande,"Self-referral, referral from family, friends, etc.; no other agency/institution involved.global.demande",Other.global.demande,Not known / missing.global.demande,Total.global.demande
0,Austria,2022,4709,1124,13,440,75,287,40,17,...,1124,337,23,30,275,19,353,54,33,1124
1,Belgium,2022,12144,3501,142,949,776,652,723,50,...,3501,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>
2,Bulgaria,2022,1879,109,4,19,21,53,11,1,...,109,28,1,4,3,0,63,9,1,109
3,Croatia *,2022,2315,478,53,102,145,135,7,8,...,478,119,20,0,95,<NA>,197,32,15,478
4,Cyprus,2022,884,340,17,116,77,98,5,16,...,340,139,0,12,10,1,104,68,6,340


#### 5.1.2 Même procédé sur les valeurs manquantes pour table_infractions_simple :
1) On **uniformise** les valeurs manquantes/aberrantes
2) On crée une ligne qui affiche une **fonction de comptage des valeurs manquantes**

In [299]:
warnings.filterwarnings('ignore')

### 1) Uniformisation des valeurs manquantes/aberrantes
for col in table_infractions_simple.columns:
    if col != 'Country':  # On exclut la colonne 'Country'
        table_infractions_simple[col] = pd.to_numeric(table_infractions_simple[col], errors='coerce')  # Coerce transforme les valeurs non convertibles en NaN

### 2) On crée la nouvelle ligne
Nb_valeurs_manquantes = table_infractions_simple.isna().sum() # On compte le nombre de NaN dans chaque colonne du dataframe "table_infractions_simple"
table_infractions_simple.loc['Nb_valeurs_manquantes'] = Nb_valeurs_manquantes # On ajoute une ligne contenant cette variable au dataframe

# On affiche le dataframe pour vérifier que l'opération a fonctionné correctement
table_infractions_simple.head()

,Country,2022.0.cannabis.conso_x,2021.0.cannabis.conso_x,2020.0.cannabis.conso_x,2019.0.cannabis.conso_x,2018.0.cannabis.conso_x,2017.0.cannabis.conso_x,2016.0.cannabis.conso_x,2015.0.cannabis.conso_x,2014.0.cannabis.conso_x,...,2016.0.cannabis.ventes,2015.0.cannabis.ventes,2014.0.cannabis.ventes,2013.0.cannabis.ventes,2012.0.cannabis.ventes,2011.0.cannabis.ventes,2010.0.cannabis.ventes,2009.0.cannabis.ventes,2008.0.cannabis.ventes,2007.0.cannabis.ventes
0,Austria,19909.0,19919.0,22778.0,24618.0,24680.0,28890.0,23977.0,20456.0,21272.0,...,7372.0,7086.0,6705.0,1595.0,1409.0,1428.0,1454.0,1460.0,1261.0,1833.0
1,Belgium,29572.0,28857.0,31540.0,31688.0,30148.0,29916.0,27080.0,28237.0,29825.0,...,6316.0,6460.0,6637.0,6074.0,5942.0,8341.0,8574.0,8892.0,9047.0,8200.0
2,Bulgaria,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3465.0,...,NaN,NaN,790.0,1424.0,800.0,625.0,627.0,641.0,438.0,314.0
3,Croatia,5159.0,5080.0,4810.0,5689.0,6788.0,6253.0,5493.0,4268.0,4866.0,...,1719.0,1620.0,1399.0,1799.0,1442.0,1790.0,1746.0,1519.0,931.0,1199.0
4,Cyprus,563.0,512.0,606.0,692.0,862.0,703.0,643.0,690.0,799.0,...,106.0,142.0,88.0,127.0,124.0,88.0,117.0,129.0,NaN,128.0


### 5.2 Lisibilité : on crée une table des différentes variables présentes dans les dataframes
*Les dataframes concernés sont table_infractions_simple et table_soins_simple*

**Comment va-t-on procéder ?**
On va créer une liste de chaque variable en subdivisant à l'aide de *.detail*

#### 5.2.1 Pour la table *table_infractions_simple*

*Les listes servent en elles-mêmes de balise pour se repérer dans les tables auxquelles elles font référence*.

In [313]:
Var_infractions_1 = ["Consommation", "Infractions", "Production", "Ventes"]
Var_infractions_2 = ["Cannabis", "Global"]
# NB1 : il suffit de croiser les deux listes (ex : .cannabis.consommation)
# NB2 : pour avoir l'année (format : '20XX'), il faut rajouter en prefixe '20XX.0.'
### Exemple : 2012.0.Infractions.Cannabis
table_soins_simple.head()

,Country,Year of Treatment,All substances (global),Cannabis (global),2022.0.global.conso_x,2021.0.global.conso_x,2020.0.global.conso_x,2019.0.global.conso_x,2018.0.global.conso_x,2017.0.global.conso_x,...,Total.global.age_premiere_conso,Court / probation / police.global.demande,General practitioner.global.demande,Other drug treatment centre.global.demande,"Other health, medical or social service.global.demande",Educational services.global.demande,"Self-referral, referral from family, friends, etc.; no other agency/institution involved.global.demande",Other.global.demande,Not known / missing.global.demande,Total.global.demande
0,Austria,2022.0,4709.0,1124.0,NaN,NaN,NaN,NaN,NaN,NaN,...,1124.0,337.0,23.0,30.0,275.0,19.0,353.0,54.0,33.0,1124.0
1,Belgium,2022.0,12144.0,3501.0,40714.0,38377.0,41194.0,42116.0,40688.0,38573.0,...,3501.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Bulgaria,2022.0,1879.0,109.0,1874.0,2153.0,1121.0,NaN,NaN,NaN,...,109.0,28.0,1.0,4.0,3.0,0.0,63.0,9.0,1.0,109.0
3,Croatia *,2022.0,2315.0,478.0,NaN,NaN,NaN,NaN,NaN,NaN,...,478.0,119.0,20.0,0.0,95.0,NaN,197.0,32.0,15.0,478.0
4,Cyprus,2022.0,884.0,340.0,762.0,669.0,759.0,858.0,1008.0,811.0,...,340.0,139.0,0.0,12.0,10.0,1.0,104.0,68.0,6.0,340.0


#### 5.2.2 Pour la table *table_soins_simple*

En plus de listes similaires à celles faites pour *table_infractions_simples*, la multiplication **pour cette table** du nombre de variable (au-delà des simples années) nous pousse - à l'aide d'une boucle *for* - de créer une liste des **modalités** existantes pour les variables **présentes dans Var_soins_1**.

In [301]:
Var_soins_1 = ['.pro','.sex', '.sdf','.lvl_educ','.frqce_conso','.enfants','.menage','.age_trait','.age_premiere_conso','.demande']
Var_soins_2 = ['cannabis', 'global']

### chaque élement de Var_soins_1 est une table de base qui contient elle même d'autres variables
# On va donc imprimer la liste de ces variables, il suffira donc de procéder par arborescence : on choisit le thème qu'on veut étudier dans la liste puis on regarde les variables disponibles à ce sujet
dossier = "/home/onyxia/work/Projet_Python_2A/Data/EUDA/Cannabis détaillé"
for j in Var_soins_1 : 
    fichiers = [f for f in os.listdir(dossier) if j in f and f.endswith('.xlsx')]
    # On ajoute le suffixe aux colonnes souhaitées
    for fichier in fichiers:
        df = pd.read_excel(chemin_fichier)

colonnes_par_soin = {}

for soin in Var_soins_1:
    colonnes = [col for col in table_soins_simple.columns if soin in col]
    colonnes_par_soin[soin] = colonnes

print (colonnes_par_soin)

{'.pro': ['Occasionally employed.global.pro', 'Regularly employed.global.pro', 'Student.global.pro', 'Unemployed / discouraged.global.pro', 'Receiving social benefits / pensioners / house-makers / disable.global.pro', 'Other.global.pro', 'Not known / missing.global.pro', 'Total.global.pro'], '.sex': ['All.global.sex', 'Males.global.sex', 'Females.global.sex', 'Not known / missing.global.sex'], '.sdf': ['Stable accommodation.global.sdf', 'Unstable accommodation and/or homeless.global.sdf', 'In detention.global.sdf', 'Other.global.sdf', 'Not known / missing.global.sdf', 'Total.global.sdf'], '.lvl_educ': ['Never went to school / never completed primary school (ISCED 0).global.lvl_educ', 'Primary level of education (ISCED 1).global.lvl_educ', 'Secondary level of education (ISCED 2 and ISCED 3).global.lvl_educ', 'Higher education (ISCED 4 to ISCED 6).global.lvl_educ', 'Not known / missing.global.lvl_educ', 'Total.global.lvl_educ'], '.frqce_conso': ['Daily.global.frqce_conso', '4 to 6 days p

**On voit ci-dessus *print* les modalités en question**

## 6. Dataframe sur les données macroéconomiques des pays de l'union européenne élargie
*On travaille sur des jeux de données issues de l'INSEE et d'EUROSTAT*

Ici, l'idée est de fournir un tableau général de la situation de chaque pays étudié. 
Les jeux de données que nous traitons concernent donc des variables à proprement **économiques** (PIB, Revenu médian par pays) mais également **socio-économiques** (IDH notamment).
Par ailleurs, on avons choisi de là encore prendre des données sur plusieurs années (*2011-2022*) afin de données du recul sur les observations, d'offrir plus de perspectives en matière de statistiques descriptives mais également d'avoir plus de données pour *nourrir* nos modèles de prédiction.

### 6.1 Importation des données au format pandas

Les données sont initialement enregistrées au format **excel**, on utilise donc *pandas* pour les importer en **dataframes** et pouvoir travailler dessus.

#### ***Données sur le PIB***

In [302]:
import pandas as pd

# On importe au format pandas les données sur le chômage
pib = pd.read_csv('/home/onyxia/work/Projet_Python_2A/Data/Macro/estat_tipsna40_filtered_en.csv')
pib.drop(columns=['DATAFLOW','LAST UPDATE','freq','na_item','unit','OBS_FLAG'],inplace=True)

pib = pib[pib['TIME_PERIOD'] == 2022]

pib.drop(columns='TIME_PERIOD',inplace=True)

pib.rename(columns={'geo':'Country','OBS_VALUE':'PIB_r_hab'}, inplace=True)

pib.head()

,Country,PIB_r_hab
27,Austria,42540
56,Belgium,40180
85,Bulgaria,8660
114,Cyprus,28860
143,Czechia,18450


### ***Données sur l'IDH***

In [ ]:
#On importe depuis le csv
idh = pd.read_csv('/home/onyxia/work/Projet_Python_2A/Data/Macro/IDH_UE.csv')

idh.head()

,Country,IDH
0,Austria,0.916
1,Belgium,0.937
2,Bulgaria,0.795
3,Croatia,0.858
4,Cyprus,0.896


### 6.2 Que faire lorsqu'il nous manque des données pour une année particulière ?

On se retrouve pour les variables suivantes avec des données pour chaque année. Dans certains cas, on manque de données récentes mais des données plus anciennes existent.

Le problème étant que si l'on remonte de plusieurs années en arrière afin de récupérer les données disponibles pour un pays en particulier, on prend le risque que les données voulues ne soient alors plus disponible pour un autre pays qui ne disposerait que de données récentes et pas anciennes par exemple.

Puisque notre objectif ici est avant tout de **minimiser le nombre de valeur manquantes**, en effet notre *dataset* est déjà très petit et l'on ne peut pas se permettre de supprimer beaucoup de lignes, on décide alors de ne garder pour chaque pays que **la donnée la plus récente disponible** pour chaque variable.

Cela a le désavantage de comparer des données qui ne proviennent pas de la même année, mais c'est une approximation que l'on se permet afin de garder le maximum de ligne possible dans notre *dataframe*

On met cela en oeuvre à l'aide d'une simple boucle *for* que l'on arrête au bon moment.

#### ***Données sur le chômage***

In [304]:
import pandas as pd

# On importe au format pandas les données sur le chômage
chomage = pd.read_csv('/home/onyxia/work/Projet_Python_2A/Data/Macro/estat_lfsa_ugadra_filtered_en.csv')
chomage.drop(columns=['DATAFLOW','LAST UPDATE','freq','age','sex','regis_es','duration','TIME_PERIOD','unit','OBS_FLAG'],inplace=True)

chomage.rename(columns={'geo':'Country','OBS_VALUE':'tx_chomage'}, inplace=True)

chomage_filtre = pd.DataFrame(columns=['Country','tx_chomage'])

# On fait en sorte de récupérer à chaque fois la donnée la plus récente disponible
for index in chomage.index:
    country = chomage.at[index, 'Country']
    last_buffer = buffer
    buffer = chomage.loc[index].tolist()
    chomage_filtre.loc[country] = buffer
    if pd.isna(chomage_filtre.at[country,'tx_chomage']) == True:
        chomage_filtre.loc[country] = last_buffer
        buffer = last_buffer

chomage_filtre.reset_index(inplace=True)
chomage_filtre.drop(columns='index', inplace=True)
chomage_filtre.drop_duplicates(inplace=True)

chomage_filtre.head()

,Country,tx_chomage
0,Austria,42.7
1,Bosnia and Herzegovina,24.7
2,Belgium,16.1
4,Switzerland,23.9
5,Cyprus,22.0


#### ***Données sur la délinquance***

On ne garde que le pourcentage de violences, délinquance et vandalisme dans les grandes villes (c'est la population en soi la plus intéressante)

In [305]:
# On importe au format pandas les données sur le chômage
delinq = pd.read_csv('/home/onyxia/work/Projet_Python_2A/Data/Macro/estat_ilc_mddw06_filtered_en.csv')
delinq.drop(columns=['DATAFLOW','LAST UPDATE','freq','incgrp','TIME_PERIOD','deg_urb','unit','OBS_FLAG'],inplace=True)

delinq.rename(columns={'geo':'Country','OBS_VALUE':'delinquance_ville%'}, inplace=True)

delinq_filtre = pd.DataFrame(columns=['Country','delinquance_ville%'])

# On fait en sorte de récupérer à chaque fois la donnée la plus récente disponible
for index in delinq.index:
    country = delinq.at[index, 'Country']
    last_buffer = buffer
    buffer = delinq.loc[index].tolist()
    delinq_filtre.loc[country] = buffer
    if pd.isna(delinq_filtre.at[country,'delinquance_ville%']) == True:
        delinq_filtre.loc[country] = last_buffer
        buffer = last_buffer

delinq_filtre.reset_index(inplace=True)
delinq_filtre.drop(columns='index', inplace=True)
delinq_filtre.drop_duplicates(inplace=True)

delinq_filtre.head()

,Country,delinquance_ville%
0,Albania,0.8
1,Austria,10.9
2,Belgium,29.6
3,Bulgaria,33.1
4,Switzerland,9.9


#### ***Données sur les dépenses sociales***

Dépenses totales de la sécurité sociale, en millions d'euros.

In [306]:
# On importe au format pandas les données sur le chômage
secu = pd.read_csv('/home/onyxia/work/Projet_Python_2A/Data/Macro/estat_spr_exp_type_filtered_en.csv')
secu.drop(columns=['DATAFLOW','LAST UPDATE','freq','unit','TIME_PERIOD','OBS_FLAG','spdeps'],inplace=True)

secu.rename(columns={'geo':'Country','OBS_VALUE':'depenses_soc'}, inplace=True)

secu_filtre = pd.DataFrame(columns=['Country','depenses_soc'])


# On fait en sorte de récupérer à chaque fois la donnée la plus récente disponible
for index in secu.index:
    country = secu.at[index, 'Country']
    last_buffer = buffer
    buffer = secu.loc[index].tolist()
    secu_filtre.loc[country] = buffer
    if pd.isna(secu_filtre.at[country,'depenses_soc']) == True:
        secu_filtre.loc[country] = last_buffer
        buffer = last_buffer

secu_filtre.reset_index(inplace=True)
secu_filtre.drop(columns='index', inplace=True)
secu_filtre.drop_duplicates(inplace=True)

secu_filtre.head()

,Country,depenses_soc
0,Albania,2174.94
1,Austria,136092.99
2,Bosnia and Herzegovina,4382.25
3,Belgium,160905.04
4,Bulgaria,16049.73


#### ***Taux de risque de pauvreté au travail***

In [307]:
# On importe au format pandas les données sur le chômage
pauvrete = pd.read_csv('/home/onyxia/work/Projet_Python_2A/Data/Macro/estat_tesov110_filtered_en.csv')
pauvrete.drop(columns=['DATAFLOW','LAST UPDATE','freq','unit','OBS_FLAG','TIME_PERIOD','wstatus','age','sex'],inplace=True)

pauvrete.rename(columns={'geo':'Country','OBS_VALUE':'risque_pauvrete'}, inplace=True)

pauvrete_filtre = pd.DataFrame(columns=['Country','risque_pauvrete'])

# On fait en sorte de récupérer à chaque fois la donnée la plus récente disponible
for index in secu.index:
    country = pauvrete.at[index, 'Country']
    last_buffer = buffer
    buffer = pauvrete.loc[index].tolist()
    pauvrete_filtre.loc[country] = buffer
    if pd.isna(pauvrete_filtre.at[country,'risque_pauvrete']) == True:
        pauvrete_filtre.loc[country] = last_buffer
        buffer = last_buffer

pauvrete_filtre.reset_index(inplace=True)
pauvrete_filtre.drop(columns='index', inplace=True)
pauvrete_filtre.drop_duplicates(inplace=True)

pauvrete_filtre.head()

,Country,risque_pauvrete
0,Albania,12.6
1,Austria,7.6
2,Belgium,4.7
3,Bulgaria,11.4
4,Switzerland,9.0


### 6.3 Merge final des données macro

On rencontre un dernier problème: il manque certains pays dans les tables contenant les données du taux de chômage et de pauvreté.

on décide alors de produire trois tables: une sans ces deux information, mais contenant la totalité des 27 pays membres de l'UE, ainsi qu'une seconde table ne contant en plus les informations sur le chômage pour 22 pays; et enfin un troisième table contenant toutes les informations pour seulement 15 pays.

In [308]:
#La première table
table_macro = pd.merge(pib,idh,how='inner',on='Country')
table_macro = pd.merge(table_macro,secu_filtre,how='inner',on='Country')
table_macro = pd.merge(table_macro,delinq_filtre,how='inner',on='Country')

# La deuxième table
table_macro_loss = pd.merge(table_macro,chomage_filtre,how='inner',on='Country')

#La troisième table
table_macro_loss_2 = pd.merge(table_macro_loss,pauvrete_filtre,how='inner',on='Country')

table_macro

,Country,PIB_r_hab,IDH,depenses_soc,delinquance_ville%
0,Austria,42540,0.916,136092.99,10.9
1,Belgium,40180,0.937,160905.04,29.6
2,Bulgaria,8660,0.795,16049.73,33.1
3,Cyprus,28860,0.896,5827.16,6.1
4,Czechia,18450,0.889,57227.63,15.8
5,Germany,39820,0.942,1186698.50,17.9
6,Denmark,54310,0.948,107268.95,16.2
7,Estonia,19240,0.890,5736.01,5.8
8,Greece,18310,0.887,50179.43,28.7
9,Spain,25380,0.905,355326.66,22.9
